In [1]:
# -*- coding: utf-8 -*-
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tensorflow as tf
import struct 
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt

import argparse



from datetime import datetime
import time

import cifar10

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### 这是客户端本地项目路径  映射到   /code

In [3]:
!ls

BUILD	     cifar10_eval.py		 lnvfile_notebook.yml
README.md    cifar10_input.py		 lnvfile_tensorboard.yml
__init__.py  cifar10_input_test.py	 predict.es.ipynb
__pycache__  cifar10_multi_gpu_train.py  train.cifar.ipynb
ci	     cifar10_train.py
cifar10.py   lnvfile.yml


In [4]:
!ls cifar10_data/cifar-10-batches-bin

ls: cannot access 'cifar10_data/cifar-10-batches-bin': No such file or directory


In [9]:
!ls /data/cifar-10-data/

cifar-10-batches-py	eval.tfrecords	 validation.tfrecords
cifar-10-python.tar.gz	train.tfrecords


In [12]:
!ls /data/cifar10

cifar-10-batches-bin  cifar-10-binary.tar.gz


In [10]:
!ls /data/cifar10/

cifar-10-batches-bin  cifar-10-binary.tar.gz


#### 下载数据库
     /data/实验名字

####  数据路径存在虚拟主机路径下：

In [19]:
!ls /data/cifar10
data_path = '/data/cifar10/'

cifar-10-batches-bin  cifar-10-binary.tar.gz


#### 模型路径 

In [16]:
!ls /outputs/ipython/cifar10

checkpoint
events.out.tfevents.1553947970.plx-notebook-e8a2924b2c404635b65477e3cbf49458-7788c5db45-tfxft
graph.pbtxt
model.ckpt-1.data-00000-of-00002
model.ckpt-1.data-00001-of-00002
model.ckpt-1.index
model.ckpt-1.meta


In [17]:
train_path_local = '/outputs/ipython/cifar10'

In [18]:
#模型参数
max_steps =100
#train_dir   = data_path_local
log_device_placement = False

log_frequency = 10

In [20]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('data_dir', data_path,
                           """Path to the CIFAR-10 data dir """)

tf.app.flags.DEFINE_string('train_dir', train_path_local,
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 100000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")
tf.app.flags.DEFINE_integer('log_frequency', 10,
                            """How often to log results to the console.""")

In [21]:
def set_logging(log_level=None):
    if log_level == 'INFO':
        log_level = tf.logging.INFO
    elif log_level == 'DEBUG':
        log_level = tf.logging.DEBUG
    elif log_level == 'WARN':
        log_level = tf.logging.WARN
    else:
        log_level = 'INFO'

    tf.logging.set_verbosity(log_level)

In [22]:
set_logging('DEBUG')


In [23]:
#定义模型网络  tf.esstimator api
def train():
  """Train CIFAR-10 for a number of steps."""
  with tf.Graph().as_default():

    global_step = tf.train.get_or_create_global_step()

    # Get images and labels for CIFAR-10.
    # Force input pipeline to CPU:0 to avoid operations sometimes ending up on
    # GPU and resulting in a slow down.
    with tf.device('/cpu:0'):
      images, labels = cifar10.distorted_inputs()

    # Build a Graph that computes the logits predictions from the
    # inference model.
    logits = cifar10.inference(images)

    # Calculate loss.
    loss = cifar10.loss(logits, labels)

    # Build a Graph that trains the model with one batch of examples and
    # updates the model parameters.
    train_op = cifar10.train(loss, global_step)

    class _LoggerHook(tf.train.SessionRunHook):
      """Logs loss and runtime."""

      def begin(self):
        self._step = -1
        self._start_time = time.time()

      def before_run(self, run_context):
        self._step += 1
        return tf.train.SessionRunArgs(loss)  # Asks for loss value.

      def after_run(self, run_context, run_values):
        if self._step % FLAGS.log_frequency == 0:
          current_time = time.time()
          duration = current_time - self._start_time
          self._start_time = current_time

          loss_value = run_values.results
          examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
          sec_per_batch = float(duration / FLAGS.log_frequency)

          format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                        'sec/batch)')
          print (format_str % (datetime.now(), self._step, loss_value,
                               examples_per_sec, sec_per_batch))

    with tf.train.MonitoredTrainingSession(
        checkpoint_dir=FLAGS.train_dir,
        hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
               tf.train.NanTensorHook(loss),
               _LoggerHook()],
        config=tf.ConfigProto(
            log_device_placement=FLAGS.log_device_placement)) as mon_sess:
      while not mon_sess.should_stop():
        mon_sess.run(train_op)



In [24]:
#解决jupyter下cifar10 报错 UnrecognizedFlagError: Unknown command line flag 'f'
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [25]:
cifar10.maybe_download_and_extract()

In [28]:
train()

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
INFO:tensorflow:Summary name local3/weight_loss (raw) is illegal; using local3/weight_loss__raw_ instead.
INFO:tensorflow:Summary name local4/weight_loss (raw) is illegal; using local4/weight_loss__raw_ instead.
INFO:tensorflow:Summary name cross_entropy (raw) is illegal; using cross_entropy__raw_ instead.
INFO:tensorflow:Summary name total_loss (raw) is illegal; using total_loss__raw_ instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /outputs/ipython/cifar10/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /outputs/ipython/cifar10/model.ckpt.
2019-03-31 11:18:42.941694: step 0, loss = 4.68 (338.4 examples/sec; 0.378 sec/batch)
2019-03-31 11:18:47.831151: step 10, loss = 4.59 (261.8 examples/sec; 0.489 sec/batch)
2019-03-31 11:18:52.554852: step 20, loss = 4.52 (271.0 examples/sec; 0.472 sec/batch)
2019-03-31 11:18:58.043982: step 30, loss

KeyboardInterrupt: 

In [29]:
!ls {train_path_local}

checkpoint
events.out.tfevents.1554031094.plx-notebook-16dd8ac70e9543e58661a3f8553e292a-99f4b4ff5-kjvnj
graph.pbtxt
model.ckpt-1.data-00000-of-00002
model.ckpt-1.data-00001-of-00002
model.ckpt-1.index
model.ckpt-1.meta
model.ckpt-2.data-00000-of-00002
model.ckpt-2.data-00001-of-00002
model.ckpt-2.index
model.ckpt-2.meta
